# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'fruitsDB', 'local', 'met', 'travel_DB', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66e5eb11da43de1430e0ab50'),
 'geocode': {'latitude': None, 'longitude': None},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'longitude': None,


In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
penang_flavours={'BusinessName': 'Penang Flavours', 'BusinessType':'Resturant/Cafe/Canteen', 
                 'AddressLine1': 'Greenwhich'}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavours)

InsertOneResult(ObjectId('66f2344ee8013a2b68efa8eb'), acknowledged=True)

In [10]:
# Check that the new restaurant was inserted
establishments.find_one({'BusinessName':'Penang Flavours'})

{'_id': ObjectId('66e5ee2d1901f9927f3a7d5d'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Resturant',
 'BusinessTypeID': 1,
 'AddressLine1': 'Greenwhich',
 'longitude': None,
 'latitude': None,
 'RatingValue': None,
 'geocode': {'longitude': None, 'latitude': None}}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType':'Restaurant/Cafe/Canteen'}
fields = {'BusinessType', 'BusinessTypeID'}
results= list(establishments.find(query,fields))
pprint(results)

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('66e5eb11da43de1430e0ab50')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('66e5eb11da43de1430e0ab59')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('66e5eb11da43de1430e0ab5d')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('66e5eb11da43de1430e0ab61')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('66e5eb11da43de1430e0ab62')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('66e5eb11da43de1430e0ab77')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('66e5eb11da43de1430e0ab7a')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('66e5eb11da43de1430e0ab80')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_

3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    penang_flavours,
    {'$push':
         {'BusinessTypeID': 1}
    }
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [13]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName':'Penang Flavours'})

{'_id': ObjectId('66e5ee2d1901f9927f3a7d5d'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Resturant',
 'BusinessTypeID': 1,
 'AddressLine1': 'Greenwhich',
 'longitude': None,
 'latitude': None,
 'RatingValue': None,
 'geocode': {'longitude': None, 'latitude': None}}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
establishments.count_documents(query)

0

In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [16]:
# Check if any remaining documents include Dover
results = establishments.find(query)
for result in results:
    print(result)

In [17]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('66e5eb11da43de1430e0ab50'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': None, 'latitude': None},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [27]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set': {'geocode.longitude' : {'$toDouble':'$geocode.longitude'},
                                          'geocode.latitude' : {'$toDouble':'$geocode.latitude'}
                                         }
                                }])


UpdateResult({'n': 38793, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [28]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [29]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{'$set': {"RatingValue" : {'$toInt':'$RatingValue'}}} ])

UpdateResult({'n': 38793, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [30]:
# Check that the coordinates and rating value are now numbers
[type for type in establishments.find({}, ['geocode.longitude', 'geocode.latitude', 'RatingValue'])]

[{'_id': ObjectId('66e5eb11da43de1430e0ab50'),
  'RatingValue': 5,
  'geocode': {'longitude': None, 'latitude': None}},
 {'_id': ObjectId('66e5eb11da43de1430e0ab52'),
  'RatingValue': 4,
  'geocode': {'longitude': None, 'latitude': None}},
 {'_id': ObjectId('66e5eb11da43de1430e0ab57'),
  'RatingValue': 5,
  'geocode': {'longitude': None, 'latitude': None}},
 {'_id': ObjectId('66e5eb11da43de1430e0ab58'),
  'RatingValue': 5,
  'geocode': {'longitude': None, 'latitude': None}},
 {'_id': ObjectId('66e5eb11da43de1430e0ab59'),
  'RatingValue': 4,
  'geocode': {'longitude': None, 'latitude': None}},
 {'_id': ObjectId('66e5eb11da43de1430e0ab5a'),
  'RatingValue': 5,
  'geocode': {'longitude': None, 'latitude': None}},
 {'_id': ObjectId('66e5eb11da43de1430e0ab5b'),
  'RatingValue': 5,
  'geocode': {'longitude': None, 'latitude': None}},
 {'_id': ObjectId('66e5eb11da43de1430e0ab5c'),
  'RatingValue': 5,
  'geocode': {'longitude': None, 'latitude': None}},
 {'_id': ObjectId('66e5eb11da43de1430e0a